# PYTORCH CNN Classifier

In [1]:
DATASET = 'none'
VOCAB_SIZE = 0
TOKENIZER = 'none'
KMER = 0

In [2]:
# Parameters
DATASET = "human_nontata_promoters"
TOKENIZER = "kmer"
KMER = 3


In [3]:
print(DATASET, VOCAB_SIZE, TOKENIZER, KMER)

human_nontata_promoters 0 kmer 3


## Config

In [4]:
import torch
from torch.utils.data import DataLoader

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from glp.models import CNN
from glp.tokenizers import get_tokenizer
from glp.tokenizers.utils import build_vocab, coll_factory, check_config, check_seq_lengths

In [5]:
config = {
    "dataset": DATASET,
    "tokenizer": TOKENIZER,
    "dataset_version": 0,
    "epochs": 5,
    "batch_size": 32,
    "use_padding": True,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [6]:
train_dset = get_dataset(config["dataset"], 'train')

## Tokenizer and vocab

In [7]:
tokenizer = get_tokenizer(config['tokenizer'])
tokenizer.train(train_dset=train_dset, vocab_size=VOCAB_SIZE, kmer=KMER)
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 66
{'GTT': 63, 'CCG': 61, 'ACG': 60, 'AGC': 58, 'TCC': 55, 'CCA': 56, 'CAA': 54, 'CAT': 62, 'AAG': 53, 'AGT': 49, 'TAC': 48, 'GCT': 39, 'GAA': 2, 'ATG': 46, 'GTC': 57, 'CGA': 9, 'GCC': 36, 'GGC': 31, 'TTT': 30, 'TAT': 47, 'ATC': 29, 'CGC': 35, 'CGG': 13, 'CTT': 25, 'CTC': 28, 'TGT': 24, 'CTA': 23, 'TTG': 38, 'GGA': 21, 'AAT': 7, 'ACC': 22, '<pad>': 65, 'CAG': 1, 'TCA': 44, 'TTA': 51, 'TGG': 19, 'GGT': 45, '<unk>': 0, 'CAC': 27, 'AGA': 17, 'TAG': 10, 'TGA': 37, 'GAG': 16, 'TAA': 43, 'TCT': 42, 'ACT': 5, 'GTG': 11, 'ATT': 14, 'TTC': 18, 'AAA': 64, 'ACA': 40, 'GCA': 12, 'GAT': 59, 'GGG': 20, 'GAC': 41, 'CCC': 34, 'AGG': 6, 'CTG': 26, 'GCG': 15, 'AAC': 4, 'GTA': 52, 'ATA': 50, 'CGT': 33, 'TGC': 32, 'CCT': 8, 'TCG': 3}


## Dataloader and batch preparation

In [8]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_tok_len = check_seq_lengths(dataset=train_dset, tokenizer=tokenizer)

# Data Loader
collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = max_tok_len)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cpu device


max_tok_len  83


In [9]:
tokenizer(train_dset[1][0])

['CAC',
 'TCT',
 'GTG',
 'TTC',
 'TTT',
 'TAA',
 'AAT',
 'CCC',
 'TCA',
 'GGT',
 'TTT',
 'ATG',
 'TTT',
 'TAT',
 'TGC',
 'TAC',
 'CAG',
 'AGT',
 'CTG',
 'CCT',
 'CCC',
 'TGA',
 'GGT',
 'TCT',
 'TGT',
 'ATA',
 'GAC',
 'TAG',
 'TTA',
 'TTT',
 'CCC',
 'TCT',
 'GTA',
 'AAG',
 'AAG',
 'CTG',
 'TTC',
 'TAT',
 'TCG',
 'TTC',
 'TCG',
 'CCT',
 'GGT',
 'TTG',
 'GAA',
 'CAA',
 'ACT',
 'GAA',
 'CAC',
 'TTC',
 'CAA',
 'AGG',
 'AGG',
 'CAG',
 'TCC',
 'TTG',
 'CAG',
 'CCT',
 'TGT',
 'CTC',
 'CTT',
 'CCA',
 'CTC',
 'CCC',
 'TCC',
 'TCC',
 'CCA',
 'CAG',
 'TCC',
 'TGG',
 'CTG',
 'GAG',
 'CAG',
 'CGA',
 'GTC',
 'TGT',
 'CGA',
 'TCC',
 'CAG',
 'GCC',
 'AGA',
 'GAC',
 'AAG']

## Model

In [10]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=max_tok_len
).to(device)

## Training

In [11]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/Users/katarina/Documents/repositories/GitHub/GLP/src/glp/tokenizers/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 78.0%, Avg loss: 0.622050 

Epoch 1


Train metrics: 
 Accuracy: 80.4%, Avg loss: 0.612711 

Epoch 2


Train metrics: 
 Accuracy: 82.5%, Avg loss: 0.604389 

Epoch 3


Train metrics: 
 Accuracy: 79.3%, Avg loss: 0.609369 

Epoch 4


Train metrics: 
 Accuracy: 84.1%, Avg loss: 0.595688 



## Testing

In [12]:
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

p  4119 ; tp  3052.838479042053 ; fp  815.2843484993386
recall  0.7411601065894764 ; precision  0.789229974111877
num_batches 283
correct 7170
size 9034
Test metrics: 
 Accuracy: 0.793668, F1 score: 0.764440, Avg loss: 0.618350 



(0.7936683639583795, 0.7644400981327547)